# Desafio Python e E-mail

### Descri��o

Digamos que voc� trabalha em uma ind�stria e est� respons�vel pela �rea de intelig�ncia de neg�cio.

Todo dia, voc�, a equipe ou at� mesmo um programa, gera um report diferente para cada �rea da empresa:
- Financeiro
- Log�stica
- Manuten��o
- Marketing
- Opera��es
- Produ��o
- Vendas

Cada um desses reports deve ser enviado por e-mail para o Gerente de cada �rea.

Crie um programa que fa�a isso automaticamente. A rela��o de Gerentes (com seus respectivos e-mails) e �reas est� no arquivo 'Enviar E-mails.xlsx'.

Dica: Use o pandas read_excel para ler o arquivo dos e-mails que isso vai facilitar.

In [1]:
import pandas as pd
from pathlib import Path

def formatar_planilha():
    # Lê a planilha do Excel em um DataFrame
    df_emails = pd.read_excel(Path(r'Arquivos Excel/Enviar E-mails.xlsx'))
    
    # Itera sobre as colunas no DataFrame
    for coluna in df_emails.columns:
        # Verifica se a coluna é 'E-mail'
        if coluna == 'E-mail':
            # Gera e atribui endereços de e-mail com base no índice
            df_emails[coluna] = [f'emailqualquer+{i+1}@gmail.com' for i in range(len(df_emails))]

    # Salva o DataFrame modificado de volta à planilha do Excel
    df_emails.to_excel(r'Arquivos Excel/Enviar E-mails.xlsx', index=False)
    
    # Retorna o DataFrame modificado
    return df_emails
 
def caminho_arquivos(planilha_formatada):
    # Extrai a coluna 'Relatório' do DataFrame formatado
    relatorio_coluna = planilha_formatada['Relatório']
    
    # Inicializa um dicionário vazio para armazenar associações de e-mail e arquivo
    dicionario_emails = {}
    
    # Obtém o diretório de trabalho atual
    caminho = Path.cwd()
    
    # Itera sobre os arquivos no diretório atual
    for i, arquivo in enumerate(caminho.iterdir()):
        # Pula os dois primeiros arquivos no diretório (a enumeração começa em 0)
        if i > 1:
            # Divide o nome do arquivo e a extensão
            nome_arquivo, extensao = arquivo.name.split('.')
            
            # Verifica se o nome do arquivo está presente na coluna 'Relatório'
            if nome_arquivo in relatorio_coluna.values:
                print(f'O nome do arquivo {nome_arquivo} está presente na coluna "Relatório".')
                
                # Obtém o e-mail correspondente da coluna 'E-mail'
                email_correspondente = planilha_formatada.loc[relatorio_coluna == nome_arquivo, 'E-mail'].values[0]
                
                # Cria uma entrada no dicionário com o e-mail como chave e o nome do arquivo como valor
                dicionario_emails[email_correspondente] = nome_arquivo + f'.{extensao}'
    
    # Retorna o dicionário de associações de e-mail e arquivo              
    return dicionario_emails

# Formata a planilha do Excel e obtém as associações de e-mail e arquivo     
planilha_formatada = formatar_planilha()
dicionario_emails_arquivos = caminho_arquivos(planilha_formatada)

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

def enviar_email(dicionario_emails_arquivos, planilha_formatada):  
    # Configurações para o servidor SMTP do Outlook
    servidor_smtp = 'smtp.office365.com'
    porta_smtp = 587
    msg = MIMEMultipart()

    # Configuração inicial da mensagem comum a todos os e-mails
    corpo_email = """
    <html>
    <head>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f4;
                margin: 0;
                padding: 20px;
            }
            h1 {
                color: #333;
            }
            p {
                color: #555;
            }
        </style>
    </head>
    <body>
        <h1>Teste de E-mail</h1>
        <p>Olá,</p>
        <p>Este é um exemplo de e-mail com formatação HTML.</p>
        <p>Parágrafo1</p>
        <p>Parágrafo2</p>
        <p>Obrigado!</p>
    </body>
    </html>
    """

    relatorio_coluna = planilha_formatada['Relatório']

    for email_destino, arquivo_xlsx in dicionario_emails_arquivos.items():
        arquivo_xlsx, extensao = arquivo_xlsx.split(".")
        
        if arquivo_xlsx in relatorio_coluna.values:
            # Juntando a o arquivo a extensão
            arquivo_xlsx = arquivo_xlsx + f'.{extensao}'
            
            # Cria uma nova mensagem para cada e-mail
            msg = MIMEMultipart()
            
            # Adiciona o corpo do e-mail
            corpo_mensagem = MIMEText(corpo_email, 'html')
            msg.attach(corpo_mensagem)

            # Anexa o arquivo ao e-mail
            with open(arquivo_xlsx, 'rb') as arquivo:
                anexo = MIMEApplication(arquivo.read(), Name=arquivo_xlsx)
                anexo['Content-Disposition'] = f'attachment; filename="{arquivo_xlsx}"'
                msg.attach(anexo)

            # Configura o destinatário atual
            msg['To'] = email_destino
            msg['Subject'] = "Um teste de envio de e-mail"  # Assunto do email
            msg['From'] = ''  # Substitua pelo seu endereço de e-mail do Outlook
            password = ''  # Substitua pela sua senha ou pela senha de aplicativo

            # Inicia a conexão SMTP e envia o e-mail
            with smtplib.SMTP(servidor_smtp, porta_smtp) as s:
                s.starttls()
                # Login Credentials for sending the mail
                s.login(msg['From'], password)
                s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))

            print(f'E-mail enviado para {email_destino} com o arquivo {arquivo_xlsx}')


enviar_email(dicionario_emails_arquivos, planilha_formatada)

print('Todos os e-mails enviados com sucesso!')